# Advanced Build: LinkedIn Post Generator for ML Papers

This notebook implements the Advanced Build requirement: a multi-agent LangGraph system that generates LinkedIn posts about Machine Learning papers with verification and platform-specific optimization.

## System Overview

Our system consists of three specialized teams:

1. **Paper Analysis Team**: Extracts key insights from ML papers
2. **Content Creation Team**: Generates LinkedIn-optimized posts
3. **Verification Team**: Validates accuracy and LinkedIn compliance

All orchestrated by a Meta-Supervisor for seamless workflow management.

## Dependencies and Setup

In [ ]:
import os
import getpass
from typing import Any, Callable, List, Optional, TypedDict, Union, Annotated
from pathlib import Path
import json
import re
from urllib.parse import urlparse

# Set up API keys
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY:")

# LangChain imports
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.runnables import Runnable
from langchain_core.tools import BaseTool, tool
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_community.document_loaders import ArxivLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain_core.output_parsers import StrOutputParser

# LangGraph imports
from langgraph.graph import END, StateGraph
import functools
import operator
import tiktoken

print("✅ All dependencies imported successfully!")

## Helper Functions

Reusing the helper functions from the main notebook for consistency.

In [ ]:
def agent_node(state, agent, name):
    """Create an agent node for the graph."""
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

def create_agent(
    llm: ChatOpenAI,
    tools: list,
    system_prompt: str,
) -> str:
    """Create a function-calling agent and add it to the graph."""
    system_prompt += ("\nWork autonomously according to your specialty, using the tools available to you."
    " Do not ask for clarification."
    " Your other team members (and other teams) will collaborate with you with their own specialties."
    " You are chosen for a reason!")
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_functions_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

def create_team_supervisor(llm: ChatOpenAI, system_prompt, members) -> str:
    """An LLM-based router for team supervision."""
    options = ["FINISH"] + members
    function_def = {
        "name": "route",
        "description": "Select the next role.",
        "parameters": {
            "title": "routeSchema",
            "type": "object",
            "properties": {
                "next": {
                    "title": "Next",
                    "anyOf": [
                        {"enum": options},
                    ],
                },
            },
            "required": ["next"],
        },
    }
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            (
                "system",
                "Given the conversation above, who should act next?"
                " Or should we FINISH? Select one of: {options}",
            ),
        ]
    ).partial(options=str(options), team_members=", ".join(members))
    return (
        prompt
        | llm.bind_tools(tools=[function_def], tool_choice="route")
        | JsonOutputFunctionsParser()
    )

print("✅ Helper functions defined!")

## State Definitions

Define the state structure for our multi-team system.

In [ ]:
# State for individual teams
class PaperAnalysisState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    team_members: List[str]
    next: str
    paper_data: dict
    analysis_results: dict

class ContentCreationState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    team_members: List[str]
    next: str
    analysis_results: dict
    content_draft: str
    platform_specs: dict

class VerificationState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    team_members: List[str]
    next: str
    paper_data: dict
    content_draft: str
    verification_results: dict
    final_post: str

# Main state for the entire system
class MainState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    paper_url: str
    paper_data: dict
    analysis_results: dict
    content_draft: str
    verification_results: dict
    final_post: str
    next: str

print("✅ State definitions created!")

## Paper Analysis Team

This team extracts key insights from ML papers using Arxiv tools and technical analysis.

In [ ]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini")

# Paper Analysis Tools
@tool
def extract_arxiv_id_from_url(url: Annotated[str, "Arxiv URL to extract ID from"]) -> str:
    """Extract Arxiv ID from various Arxiv URL formats."""
    patterns = [
        r'arxiv\.org/abs/(\d+\.\d+)',  # https://arxiv.org/abs/2308.08155
        r'arxiv\.org/pdf/(\d+\.\d+)',  # https://arxiv.org/pdf/2308.08155.pdf
        r'arxiv\.org/abs/([a-zA-Z-]+/\d+)',  # https://arxiv.org/abs/cs.AI/2308.08155
        r'arxiv\.org/pdf/([a-zA-Z-]+/\d+)',  # https://arxiv.org/pdf/cs.AI/2308.08155.pdf
    ]
    
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    return None

@tool
def fetch_paper_content(arxiv_id: Annotated[str, "Arxiv ID to fetch paper content"]) -> str:
    """Fetch and extract content from an Arxiv paper."""
    try:
        loader = ArxivLoader(query=arxiv_id)
        documents = loader.load()
        if documents:
            return documents[0].page_content[:5000]  # Limit for processing
        else:
            return "No content found for this Arxiv ID."
    except Exception as e:
        return f"Error fetching paper: {str(e)}"

@tool
def analyze_technical_claims(content: Annotated[str, "Paper content to analyze"]) -> str:
    """Analyze technical claims and methodology in the paper."""
    # This would use the LLM to analyze technical content
    return f"Technical analysis of paper content (first 1000 chars): {content[:1000]}..."

@tool
def assess_paper_impact(content: Annotated[str, "Paper content to assess impact"]) -> str:
    """Assess the significance and potential impact of the paper."""
    # This would use the LLM to assess impact
    return f"Impact assessment of paper content (first 1000 chars): {content[:1000]}..."

# Create Paper Analysis Agents
research_agent = create_agent(
    llm,
    [extract_arxiv_id_from_url, fetch_paper_content],
    "You are a research assistant specializing in extracting and analyzing academic papers. "
    "Your role is to fetch papers from Arxiv and extract key information for social media content creation."
)

technical_reviewer = create_agent(
    llm,
    [analyze_technical_claims],
    "You are a technical reviewer who validates and analyzes the technical claims in ML papers. "
    "Your role is to ensure accuracy and identify key technical contributions."
)

impact_assessor = create_agent(
    llm,
    [assess_paper_impact],
    "You are an impact assessor who evaluates the significance and potential impact of ML papers. "
    "Your role is to identify why this research matters and who would benefit from it."
)

# Create nodes
research_node = functools.partial(agent_node, agent=research_agent, name="Research")
technical_node = functools.partial(agent_node, agent=technical_reviewer, name="TechnicalReviewer")
impact_node = functools.partial(agent_node, agent=impact_assessor, name="ImpactAssessor")

print("✅ Paper Analysis Team created!")

## Content Creation Team

This team generates LinkedIn-optimized posts based on the paper analysis.

In [ ]:
# Content Creation Tools
@tool
def generate_linkedin_post(analysis_results: Annotated[str, "Analysis results to create LinkedIn post from"]) -> str:
    """Generate a LinkedIn post based on paper analysis results."""
    # This would use the LLM to generate LinkedIn content
    return f"LinkedIn post generated from analysis: {analysis_results[:500]}..."

@tool
def optimize_for_linkedin(content: Annotated[str, "Content to optimize for LinkedIn"]) -> str:
    """Optimize content specifically for LinkedIn platform guidelines and best practices."""
    # This would use the LLM to optimize for LinkedIn
    return f"LinkedIn optimized content: {content[:500]}..."

@tool
def add_engagement_elements(content: Annotated[str, "Content to add engagement elements to"]) -> str:
    """Add hashtags, mentions, and engagement hooks to the LinkedIn post."""
    # This would use the LLM to add engagement elements
    return f"Content with engagement elements: {content[:500]}..."

# Create Content Creation Agents
content_writer = create_agent(
    llm,
    [generate_linkedin_post],
    "You are a content writer specializing in creating engaging LinkedIn posts about academic research. "
    "Your role is to translate complex technical content into accessible, professional posts that engage the LinkedIn audience."
)

platform_specialist = create_agent(
    llm,
    [optimize_for_linkedin],
    "You are a LinkedIn platform specialist who optimizes content for LinkedIn's specific guidelines, "
    "formatting, and best practices. Your role is to ensure content meets LinkedIn's professional standards."
)

engagement_optimizer = create_agent(
    llm,
    [add_engagement_elements],
    "You are an engagement optimizer who adds hashtags, mentions, and engagement hooks to LinkedIn posts. "
    "Your role is to maximize visibility and engagement while maintaining professionalism."
)

# Create nodes
writer_node = functools.partial(agent_node, agent=content_writer, name="ContentWriter")
platform_node = functools.partial(agent_node, agent=platform_specialist, name="PlatformSpecialist")
engagement_node = functools.partial(agent_node, agent=engagement_optimizer, name="EngagementOptimizer")

print("✅ Content Creation Team created!")

## Verification Team

This team validates the accuracy and LinkedIn compliance of the generated content.

In [ ]:
# Verification Tools
@tool
def verify_technical_accuracy(paper_data: Annotated[str, "Original paper data"], content: Annotated[str, "Content to verify"]) -> str:
    """Verify that the generated content accurately represents the technical claims in the original paper."""
    # This would use the LLM to verify accuracy
    return f"Technical accuracy verification: Content checked against paper data (first 500 chars): {content[:500]}..."

@tool
def check_linkedin_compliance(content: Annotated[str, "Content to check for LinkedIn compliance"]) -> str:
    """Check if the content complies with LinkedIn's community guidelines and best practices."""
    # This would use the LLM to check compliance
    return f"LinkedIn compliance check: Content verified against platform guidelines (first 500 chars): {content[:500]}..."

@tool
def assess_content_quality(content: Annotated[str, "Content to assess quality"]) -> str:
    """Assess the overall quality, tone, and professionalism of the LinkedIn post."""
    # This would use the LLM to assess quality
    return f"Content quality assessment: Professional tone and clarity verified (first 500 chars): {content[:500]}..."

# Create Verification Agents
fact_checker = create_agent(
    llm,
    [verify_technical_accuracy],
    "You are a fact checker who verifies that social media content accurately represents the original paper. "
    "Your role is to ensure no technical inaccuracies or misrepresentations."
)

compliance_checker = create_agent(
    llm,
    [check_linkedin_compliance],
    "You are a compliance checker who ensures content meets LinkedIn's community guidelines and professional standards. "
    "Your role is to prevent violations and maintain platform appropriateness."
)

quality_assessor = create_agent(
    llm,
    [assess_content_quality],
    "You are a quality assessor who evaluates the overall quality, tone, and professionalism of LinkedIn posts. "
    "Your role is to ensure content is engaging, clear, and appropriate for the professional audience."
)

# Create nodes
fact_node = functools.partial(agent_node, agent=fact_checker, name="FactChecker")
compliance_node = functools.partial(agent_node, agent=compliance_checker, name="ComplianceChecker")
quality_node = functools.partial(agent_node, agent=quality_assessor, name="QualityAssessor")

print("✅ Verification Team created!")

## Team Supervisors

Create supervisors for each team to manage workflow within teams.

In [ ]:
# Paper Analysis Team Supervisor
paper_analysis_supervisor = create_team_supervisor(
    llm,
    "You are supervising the Paper Analysis Team. Your team members are: Research, TechnicalReviewer, ImpactAssessor. "
    "Coordinate the analysis of ML papers to extract key insights for social media content creation. "
    "Ensure comprehensive analysis before moving to content creation.",
    ["Research", "TechnicalReviewer", "ImpactAssessor"]
)

# Content Creation Team Supervisor
content_creation_supervisor = create_team_supervisor(
    llm,
    "You are supervising the Content Creation Team. Your team members are: ContentWriter, PlatformSpecialist, EngagementOptimizer. "
    "Coordinate the creation of LinkedIn posts based on paper analysis. "
    "Ensure content is engaging, professional, and optimized for LinkedIn.",
    ["ContentWriter", "PlatformSpecialist", "EngagementOptimizer"]
)

# Verification Team Supervisor
verification_supervisor = create_team_supervisor(
    llm,
    "You are supervising the Verification Team. Your team members are: FactChecker, ComplianceChecker, QualityAssessor. "
    "Coordinate the verification of generated content for accuracy and LinkedIn compliance. "
    "Ensure content meets all quality standards before final approval.",
    ["FactChecker", "ComplianceChecker", "QualityAssessor"]
)

print("✅ Team Supervisors created!")

## Build Individual Team Graphs

Create separate graphs for each team that can be used as nodes in the main graph.

In [ ]:
# Paper Analysis Graph
paper_analysis_graph = StateGraph(PaperAnalysisState)

paper_analysis_graph.add_node("Research", research_node)
paper_analysis_graph.add_node("TechnicalReviewer", technical_node)
paper_analysis_graph.add_node("ImpactAssessor", impact_node)
paper_analysis_graph.add_node("supervisor", paper_analysis_supervisor)

paper_analysis_graph.add_edge("Research", "supervisor")
paper_analysis_graph.add_edge("TechnicalReviewer", "supervisor")
paper_analysis_graph.add_edge("ImpactAssessor", "supervisor")
paper_analysis_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {
        "Research": "Research",
        "TechnicalReviewer": "TechnicalReviewer",
        "ImpactAssessor": "ImpactAssessor",
        "FINISH": END,
    },
)

paper_analysis_graph.set_entry_point("supervisor")
compiled_paper_analysis = paper_analysis_graph.compile()

# Content Creation Graph
content_creation_graph = StateGraph(ContentCreationState)

content_creation_graph.add_node("ContentWriter", writer_node)
content_creation_graph.add_node("PlatformSpecialist", platform_node)
content_creation_graph.add_node("EngagementOptimizer", engagement_node)
content_creation_graph.add_node("supervisor", content_creation_supervisor)

content_creation_graph.add_edge("ContentWriter", "supervisor")
content_creation_graph.add_edge("PlatformSpecialist", "supervisor")
content_creation_graph.add_edge("EngagementOptimizer", "supervisor")
content_creation_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {
        "ContentWriter": "ContentWriter",
        "PlatformSpecialist": "PlatformSpecialist",
        "EngagementOptimizer": "EngagementOptimizer",
        "FINISH": END,
    },
)

content_creation_graph.set_entry_point("supervisor")
compiled_content_creation = content_creation_graph.compile()

# Verification Graph
verification_graph = StateGraph(VerificationState)

verification_graph.add_node("FactChecker", fact_node)
verification_graph.add_node("ComplianceChecker", compliance_node)
verification_graph.add_node("QualityAssessor", quality_node)
verification_graph.add_node("supervisor", verification_supervisor)

verification_graph.add_edge("FactChecker", "supervisor")
verification_graph.add_edge("ComplianceChecker", "supervisor")
verification_graph.add_edge("QualityAssessor", "supervisor")
verification_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {
        "FactChecker": "FactChecker",
        "ComplianceChecker": "ComplianceChecker",
        "QualityAssessor": "QualityAssessor",
        "FINISH": END,
    },
)

verification_graph.set_entry_point("supervisor")
compiled_verification = verification_graph.compile()

print("✅ Individual team graphs compiled!")

## Meta-Supervisor and Main Graph

Create the meta-supervisor that orchestrates the entire workflow.

In [ ]:
# Helper functions for the main graph
def get_last_message(state):
    """Get the last message from the state."""
    return {"messages": [state["messages"][-1]]}

def join_graph(state):
    """Join the graph results back to the main state."""
    return {"messages": [state["messages"][-1]]}

# Meta-Supervisor
meta_supervisor = create_team_supervisor(
    llm,
    "You are the Meta-Supervisor coordinating the LinkedIn Post Generation System. "
    "Your teams are: Paper Analysis Team, Content Creation Team, Verification Team. "
    "Coordinate the workflow: 1) Analyze the paper, 2) Create LinkedIn content, 3) Verify and finalize. "
    "Ensure each team completes their work before moving to the next phase.",
    ["Paper Analysis Team", "Content Creation Team", "Verification Team"]
)

# Main Graph
main_graph = StateGraph(MainState)

# Add team graphs as nodes
main_graph.add_node("Paper Analysis Team", get_last_message | compiled_paper_analysis | join_graph)
main_graph.add_node("Content Creation Team", get_last_message | compiled_content_creation | join_graph)
main_graph.add_node("Verification Team", get_last_message | compiled_verification | join_graph)
main_graph.add_node("supervisor", meta_supervisor)

# Add edges
main_graph.add_edge("Paper Analysis Team", "supervisor")
main_graph.add_edge("Content Creation Team", "supervisor")
main_graph.add_edge("Verification Team", "supervisor")
main_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {
        "Paper Analysis Team": "Paper Analysis Team",
        "Content Creation Team": "Content Creation Team",
        "Verification Team": "Verification Team",
        "FINISH": END,
    },
)

main_graph.set_entry_point("supervisor")
compiled_main_graph = main_graph.compile()

print("✅ Main graph compiled successfully!")

## User Interface and Testing

Create a simple interface for users to input paper URLs and get LinkedIn posts.

In [ ]:
def generate_linkedin_post_for_paper(paper_url: str, tone: str = "professional") -> dict:
    """
    Generate a LinkedIn post for a given ML paper.
    
    Args:
        paper_url: URL of the Arxiv paper
        tone: Desired tone (professional, casual, technical)
    
    Returns:
        Dictionary containing the generated post and metadata
    """
    
    # Initialize the graph with the paper URL
    initial_state = {
        "messages": [
            HumanMessage(
                content=f"Generate a LinkedIn post for the paper at {paper_url}. "
                f"Use a {tone} tone. Ensure the post is engaging, accurate, and follows LinkedIn best practices."
            )
        ],
        "paper_url": paper_url,
        "paper_data": {},
        "analysis_results": {},
        "content_draft": "",
        "verification_results": {},
        "final_post": "",
        "next": ""
    }
    
    # Run the graph
    results = []
    for step in compiled_main_graph.stream(initial_state, {"recursion_limit": 50}):
        if "__end__" not in step:
            results.append(step)
    
    # Extract the final post
    final_state = results[-1] if results else initial_state
    
    return {
        "paper_url": paper_url,
        "tone": tone,
        "final_post": final_state.get("final_post", "Post generation in progress..."),
        "verification_results": final_state.get("verification_results", {}),
        "workflow_steps": len(results),
        "status": "completed"
    }

# Test the system
def test_system():
    """Test the LinkedIn post generation system with a sample paper."""
    
    # Example Arxiv URL (Multi-Agent Conversation paper from the main notebook)
    test_url = "https://arxiv.org/abs/2308.08155"
    
    print("🚀 Testing LinkedIn Post Generation System")
    print(f"📄 Paper URL: {test_url}")
    print("⏳ Generating LinkedIn post...")
    print("-" * 50)
    
    try:
        result = generate_linkedin_post_for_paper(test_url, "professional")
        
        print("✅ Generation completed!")
        print(f"📊 Workflow steps: {result['workflow_steps']}")
        print(f"📝 Final post length: {len(result['final_post'])} characters")
        print("-" * 50)
        print("📱 GENERATED LINKEDIN POST:")
        print("-" * 50)
        print(result['final_post'])
        print("-" * 50)
        
        return result
        
    except Exception as e:
        print(f"❌ Error during generation: {str(e)}")
        return None

print("✅ User interface and testing functions created!")

## Run the System

Let's test our LinkedIn post generation system!

In [ ]:
# Test the complete system
test_result = test_system()

if test_result:
    print("\n🎉 Advanced Build Implementation Complete!")
    print("\n📋 System Features:")
    print("✅ Multi-agent LangGraph architecture")
    print("✅ Paper analysis with technical validation")
    print("✅ LinkedIn-optimized content generation")
    print("✅ Verification and compliance checking")
    print("✅ Meta-supervisor orchestration")
    print("✅ Professional tone and engagement optimization")
else:
    print("\n❌ System test failed. Please check the implementation.")

## Usage Instructions

### How to Use the LinkedIn Post Generator

1. **Input**: Provide an Arxiv URL for any ML paper
2. **Processing**: The system will:
   - Analyze the paper's technical content
   - Generate LinkedIn-optimized content
   - Verify accuracy and compliance
   - Produce a final, polished post
3. **Output**: Ready-to-post LinkedIn content with verification report

### Example Usage
```python
result = generate_linkedin_post_for_paper(
    paper_url="https://arxiv.org/abs/2308.08155",
    tone="professional"
)
print(result['final_post'])
```

### System Architecture

The system uses a hierarchical multi-agent approach:

1. **Paper Analysis Team** (3 agents)
   - Research: Fetches and extracts paper content
   - Technical Reviewer: Validates technical claims
   - Impact Assessor: Evaluates significance

2. **Content Creation Team** (3 agents)
   - Content Writer: Creates engaging posts
   - Platform Specialist: LinkedIn optimization
   - Engagement Optimizer: Adds hashtags and hooks

3. **Verification Team** (3 agents)
   - Fact Checker: Ensures accuracy
   - Compliance Checker: LinkedIn guidelines
   - Quality Assessor: Professional standards

4. **Meta-Supervisor**: Orchestrates the entire workflow

### Key Features

- **Technical Accuracy**: Verified against original paper
- **LinkedIn Optimization**: Platform-specific formatting and guidelines
- **Professional Tone**: Appropriate for academic/professional audience
- **Engagement Elements**: Hashtags, mentions, and engagement hooks
- **Quality Assurance**: Multiple verification layers
- **Scalable Architecture**: Easy to extend for other platforms

This implementation demonstrates advanced LangGraph capabilities with sophisticated multi-agent workflows, verification systems, and platform-specific optimization.